# PROJECT 

In [1]:
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#930157', '#009966', '#72dfd9', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
         '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#930157', '#009966', '#72dfd9', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

mapping = {
    'FP1-F7':'FP1-F7',
  'F7-T7':'F7-T7',
  'T7-P7':'T7-P7',
  'P7-O1':'P7-O1',
  'FP1-F3':'FP1-F3',
  'F3-C3':'F3-C3',
  'C3-P3':'C3-P3',
  'P3-O1':'P3-O1',
  'FP2-F4':'FP2-F4',
  'F4-C4':'F4-C4',
  'C4-P4':'C4-P4',
  'P4-O2':'P4-O2',
  'FP2-F8':'FP2-F8',
  'F8-T8':'F8-T8',
  'T8-P8-0':'T8-P8-0',
  'P8-O2':'P8-O2',
  'FZ-CZ':'FZ-CZ',
  'CZ-PZ':'CZ-PZ',
  'P7-T7':'P7-T7',
  'T7-FT9':'T7-FT9',
  'FT9-FT10':'FT9-FT10',
  'FT10-T8':'FT10-T8',
  'T8-P8-1':'T8-P8-1'
}

In [2]:
# !pip install mne
import mne
file = "./test_data/chb01_03.edf"
data = mne.io.read_raw_edf(file)
data.rename_channels(mapping)
# print(set(data.ch_names).difference(set(['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1'])))
data.set_montage("standard_1020",on_missing='ignore')
raw_data = data.get_data()
# you can get the metadata included in the file and a list of all channels:
info = data.info
channels = data.ch_names
# seizures_file = "./test_data/chb01_03_seizures.edf"
# seizures = mne.io.read_raw_edf(seizures_file)

Extracting EDF parameters from /Users/sofiyagarkot/Desktop/Signals/coursework/test_data/chb01_03.edf...
EDF file detected


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sofiyagarkot/Desktop/Signals/coursework/test_data/chb01_03.edf'

In [ ]:
required = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']

In [ ]:
channels

In [ ]:
data.plot(duration=5, n_channels=len(channels)//2)

In [ ]:
data.plot_psd(fmax=128)

In [ ]:
info

In [ ]:
channels

# Independent Component Analysis

It is not hard to see how this analogy applies to EEG/MEG analysis: there are many “microphones” (sensor channels) simultaneously recording many “instruments” (blinks, heartbeats, activity in different areas of the brain, muscular activity from jaw clenching or swallowing, etc). As long as these various source signals are statistically independent and non-gaussian, it is usually possible to separate the sources using ICA, and then re-construct the sensor signals after excluding the sources that are unwanted.

https://mne.tools/stable/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html#what-is-ica


Warning!

ICA is sensitive to low-frequency drifts and therefore requires the data to be high-pass filtered prior to fitting. Typically, a cutoff frequency of 1 Hz is recommended.

# Filtering

Спитати що якщо немає очевидних джерех побічних сигналів (електричних активностей на ЕЕГ)

In [ ]:
filt_raw = data.copy()
filt_raw.load_data().filter(l_freq=0.5, h_freq=40)
ica = mne.preprocessing.ICA(n_components=20, random_state=97)
ica.fit(filt_raw)
data.load_data()
ica.plot_sources(data)

In [ ]:
ica = mne.preprocessing.ICA(n_components=15, random_state=97)
ica.fit(filt_raw)

In [ ]:
data.load_data()
ica.plot_sources(data)

# Ентропія перестановок для передбачення початку нападу епілепсії
Оцінює хаотичність часового ряду
Міра яка зарактеризує символьну динаміку (оцінюються не числові значення а те що вони ознасають без значення того, чому це рівно)
Оцінює те, які мотиви (паттерни) знаходяться в нашому сигналі
https://drive.google.com/file/d/17yZRraKMTLqjIkbtQr0JKlru47dPIEjD/view 

Ентропія перестановок кількісно оцінює наскільки хаотичний наш сигнал. В разі, якщо значення ентропії дуже мале (або ж починається зменшуватись), тоді система починає слідувати певній впорядкованості -- стававти більш передбачуваною. 

Ентропія перестановок системи залежить від двох змінних (окрім самого сигналу):

1) Порядком ентропії -- кількість точок які ми беремо до уваги при формуванні певних патернів

2) Параметром затримки -- кількість точок (відліків сигналу) яку ми пропускаємо перш ніж "взяти до уваги" наступний елемент сигналу. Інтервал між сусідніми точками в патерні. 

Code of permutation entropy got from: https://pydoc.net/pyentrp/0.3.1/pyentrp.entropy/ 

In [ ]:
import numpy as np
import itertools
def permutation_entropy(time_series, m, delay):
    """Calculate the Permutation Entropy
 
    Args:
        time_series: Time series for analysis
        m: Order of permutation entropy
        delay: Time delay
 
    Returns:
        Vector containing Permutation Entropy
 
    Reference:
        [1] Massimiliano Zanin et al. Permutation Entropy and Its Main Biomedical and Econophysics Applications:
            A Review. http://www.mdpi.com/1099-4300/14/8/1553/pdf
        [2] Christoph Bandt and Bernd Pompe. Permutation entropy — a natural complexity
            measure for time series. http://stubber.math-inf.uni-greifswald.de/pub/full/prep/2001/11.pdf
        [3] http://www.mathworks.com/matlabcentral/fileexchange/37289-permutation-entropy/content/pec.m
    """
    n = len(time_series)
    permutations = np.array(list(itertools.permutations(range(m))))
    c = [0] * len(permutations)
 
    for i in range(n - delay * (m - 1)):
        # sorted_time_series =    np.sort(time_series[i:i+delay*m:delay], kind='quicksort')
        sorted_index_array = np.array(np.argsort(time_series[i:i + delay * m:delay], kind='quicksort'))
        for j in range(len(permutations)):
            if abs(permutations[j] - sorted_index_array).any() == 0:
                c[j] += 1
 
    c = [element for element in c if element != 0]
    p = np.divide(np.array(c), float(sum(c)))
    pe = -sum(p * np.log(p))/np.log(np.math.factorial(m))
    return pe

In [ ]:
fs = 256
window_duration = 1 
len_of_window = 1*fs
number_of_periods = len(raw_data[0])/256
first_channel_signal = raw_data[0]
m = 3
lag = 3

def get_permutation(fs, window_duration, signal, m, lag):
    number_of_periods = len(signal) // fs
    
    permutation_en = []
    
    for starting_period in range(number_of_periods):
        signal_ = signal[starting_period*fs:(starting_period+window_duration)*fs]
        entropy = permutation_entropy(signal_, m, lag)
        permutation_en.append(entropy)
        
    return permutation_en

permutations_first_channel = []
time_ = np.arange(0, number_of_periods, 1)

for starting_period in range(int(number_of_periods)):
    signal_ = first_channel_signal[starting_period*fs:(starting_period+window_duration)*fs]
    entropy = permutation_entropy(signal_, m, lag)
    permutations_first_channel.append(entropy)
     

In [ ]:
permutation_entropy(raw_data[0], 3, 1), permutation_entropy(raw_data[1], 3, 1), permutation_entropy(raw_data[3], 3, 1)

In [ ]:
import matplotlib.pyplot as plt
fig= plt.figure(figsize=(30,10))  
plt.plot(time_, get_permutation(256, 1, first_channel_signal, 3,3), COLORS[0], label = 'Channel '+str(1))
plt.xlabel('Time, s', fontsize = 18)
plt.ylabel('Entropy', fontsize = 18) 
plt.title('Permutation entropy as a function of time', weight='bold', fontsize=35)
plt.legend(fontsize='xx-large')
plt.show()

# Виділення області епілептичного нападу з довколишнім часом в х секунд. 

In [ ]:
seizure_time = [2996, 3036]
window_before = 100
window_after = 50
count_start = (seizure_time[0] - window_before)*fs
count_end = (seizure_time[1] + window_after)*fs
epilepsy_region_ch0 = raw_data[0][count_start:count_end]
epilepsy_region_ch1 = raw_data[1][count_start:count_end]
epilepsy_region_ch2 = raw_data[2][count_start:count_end]
n_counts = (seizure_time[1] + window_after)*fs - (seizure_time[0] - window_before)*fs
# time_ = np.linspace(seizure_time[0] - window_before, seizure_time[1] + window_after, n_counts)
time_ = np.arange(seizure_time[0] - window_before, seizure_time[1] + window_after, 1)
# permutation_epilepsy_region = 

fig= plt.figure(figsize=(30,10))  
plt.plot(time_, get_permutation(fs, 1, epilepsy_region_ch0, 3, 3), COLORS[0], label = 'Channel '+str(1))
plt.plot(time_, get_permutation(fs, 1, epilepsy_region_ch1, 3, 3), COLORS[4], label = 'Channel '+str(2))
plt.plot(time_, get_permutation(fs, 1, epilepsy_region_ch2, 3, 3), COLORS[2], label = 'Channel '+str(3))
plt.xlabel('Time, s', fontsize = 18)
plt.ylabel('Entropy', fontsize = 18) 
plt.title('Permutation entropy as a function of time', weight='bold', fontsize=35)
plt.legend(fontsize='xx-large')
plt.show()

for ch_indx in range(len(channels)):
    print(ch_indx, channels[ch_indx])
    epilepsy_region = raw_data[ch_indx][count_start:count_end]
    fig= plt.figure(figsize=(30,10))  
    plt.plot(time_, get_permutation(fs, 1, epilepsy_region, 3, 3), COLORS[ch_indx], label = 'Channel '+str(channels[ch_indx]))
    plt.xlabel('Time, s', fontsize = 18)
    plt.ylabel('Entropy', fontsize = 18) 
    plt.title('Permutation entropy as a function of time', weight='bold', fontsize=35)
    plt.legend(fontsize='xx-large')
    plt.show()

In [ ]:
collecting_ = []
average = []
for ch_indx in range(len(channels)):
    print(ch_indx, channels[ch_indx])
    epilepsy_region = raw_data[ch_indx][count_start:count_end]
    collecting_.append(get_permutation(fs, 1, epilepsy_region, 3, 3))
    
for a in range(len(collecting_[0])):
    value = []
    for vector in collecting_:
        value.append(vector[a])
    average.append(sum(value)/len(value))
fig= plt.figure(figsize=(30,10))  
plt.plot(time_, average, COLORS[8], label = 'Average')
plt.xlabel('Time, s', fontsize = 18)
plt.ylabel('Entropy', fontsize = 18) 
plt.title('Permutation entropy as a function of time', weight='bold', fontsize=35)
plt.legend(fontsize='xx-large')
plt.show()



Можна помітити, що з часом сигнал на певних каналах стає все більш передбачуваним, в той час як на інших каналах ентропія перестановок під час епілептичного нападу навпаки збільшуватиметься. Для того, щоб не змішувати різні тренди ентропії методом усереднення я використаю алгоритм ІСА для отримання сигналів

# Independent Component Analysis

It is not hard to see how this analogy applies to EEG/MEG analysis: there are many “microphones” (sensor channels) simultaneously recording many “instruments” (blinks, heartbeats, activity in different areas of the brain, muscular activity from jaw clenching or swallowing, etc). As long as these various source signals are statistically independent and non-gaussian, it is usually possible to separate the sources using ICA, and then re-construct the sensor signals after excluding the sources that are unwanted.

https://mne.tools/stable/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html#what-is-ica



Warning!

ICA is sensitive to low-frequency drifts and therefore requires the data to be high-pass filtered prior to fitting. Typically, a cutoff frequency of 1 Hz is recommended.


My preious assumption:
By applying the ICA I can виділити основні компоненти зміни такі як м'язева активність, ковтання, серцевий ритм). Тоді проаналізувавши ці компоненти виділити одну, чи декілька і по них поміряти ентропію перестановок, що означатиме, що певні патерни перед епілептичним нападом присутні тільки в завдяки певній активності мозку. 

# What are artifacts?
Artifacts are parts of the recorded signal that arise from sources other than the source of interest (i.e., neuronal activity in the brain). As such, artifacts are a form of interference or noise relative to the signal of interest. There are many possible causes of such interference, for example:

Environmental artifacts
Persistent oscillations centered around the AC power line frequency (typically 50 or 60 Hz)
Brief signal jumps due to building vibration (such as a door slamming)
Electromagnetic field noise from nearby elevators, cell phones, the geomagnetic field, etc.
Instrumentation artifacts
Electromagnetic interference from stimulus presentation (such as EEG sensors picking up the field generated by unshielded headphones)
Continuous oscillations at specific frequencies used by head position indicator (HPI) coils
Random high-amplitude fluctuations (or alternatively, constant zero signal) in a single channel due to sensor malfunction (e.g., in surface electrodes, poor scalp contact)
Biological artifacts
Periodic QRS-like signal patterns (especially in magnetometer channels) due to electrical activity of the heart
Short step-like deflections (especially in frontal EEG channels) due to eye movements
Large transient deflections (especially in frontal EEG channels) due to blinking
Brief bursts of high frequency fluctuations across several channels due to the muscular activity during swallowing
There are also some cases where signals from within the brain can be considered artifactual. For example, if a researcher is primarily interested in the sensory response to a stimulus, but the experimental paradigm involves a behavioral response (such as button press), the neural activity associated with the planning and executing the button press could be considered an artifact relative to signal of interest (i.e., the evoked sensory response).

## Filtering

In [ ]:
filt_raw = data.copy()
filt_raw.load_data().filter(l_freq=1., h_freq=None)
ica = mne.preprocessing.ICA(n_components=15, random_state=97)
ica.fit(filt_raw)
data.load_data()
ica.plot_sources(data)
# ica.plot_components()

In [ ]:
ica.plot_overlay(data, exclude=[0,1,2,3,5], picks='eeg')

In [ ]:
ica.plot_properties(data, picks=[0, 1])

In [ ]:
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
print(ten_twenty_montage)
# fig = ten_twenty_montage.plot(kind='3d')
# fig.gca().view_init(azim=70, elev=15)
# ten_twenty_montage.plot(kind='topomap', show_names=False)

# raw_1020 = data.copy().set_montage(ten_twenty_montage)


In [ ]:
ica.plot_properties(data, picks=[0, 1])

# Аналіз рекурентності 
Застосувати для перегляду залежностей в певній області (глянути інженірінг фіч з того пейперу з каглем)
І глянути залежності цих фіч в системі на тривалості всього сигналу, або ж тільки під час епілептичних приступів. Можливо перед ними. 

https://drive.google.com/file/d/1ISutgEutOjZuYhDHGdBGDCcgnJ_yuE6a/view


# Dispersion analysis

In [ ]:
round(np.var(raw_data[0]),100), round(np.var(raw_data[0][2996*256: 3036*256]),100)

In [ ]:
plt.plot(np.linspace(0, len(raw_data[0])//256, len(raw_data[0])), raw_data[0])

In [ ]:
np.var(raw_data[0][2996*256: 3036*256])/np.var(raw_data[0]), np.var(raw_data[1][2996*256: 3036*256])/np.var(raw_data[1]), np.var(raw_data[2][2996*256: 3036*256])/np.var(raw_data[2])

In [ ]:
from sklearn.decomposition import FastICA
type(raw_data), raw_data.shape

In [ ]:
X = raw_data.T
transformer = FastICA(n_components=7, random_state=0)
X_transformed = transformer.fit_transform(X)
X_transformed.shape, X.shape

In [ ]:
fig= plt.figure(figsize=(30,10))  
plt.plot(np.linspace(0,len(X.T[0])//256,len(X.T[0])),X.T[0])    
#     plt.ylabel('Entropy', fontsize = 18) 
#     plt.title('Permutation entropy as a function of time', weight='bold', fontsize=35)
#     plt.legend(fontsize='xx-large')
plt.show()


In [ ]:
2996-3036

In [ ]:
#seizure vs non-seizure plot
fig= plt.figure(figsize=(30,10))  
plt.plot(np.linspace(0,len(X.T[0])//256,40*256),X.T[0][1590*256 : 1630*256], COLORS[0], label = 'Non-seizure activity 1')

plt.plot(np.linspace(0,len(X.T[0])//256,40*256),X.T[0][850*256 : 890*256], COLORS[1], label = 'Non-seizure activity 2')

plt.plot(np.linspace(0,len(X.T[0])//256,40*256),X.T[0][2996*256 : 3036*256], COLORS[2], label = 'Seizure')
plt.plot(np.linspace(0,len(X.T[0])//256,40*256),X.T[0][3500*256 : 3540*256], COLORS[3], label = 'Non-seizure activity 2')

plt.legend(fontsize='xx-large')
plt.show()



In [ ]:
plt.plot(np.arange(0,len(X_transformed[5]),1),X_transformed[0])